<a href="https://colab.research.google.com/github/nicolasiovu/hackthefuture/blob/gemini/HTF_Gemini_BackEnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Colab_Notebooks/HTF_Project/hackthefuture
!ls
!git pull

/content/drive/MyDrive/Colab_Notebooks/HTF_Project/hackthefuture
backend  frontend  README.md
Already up to date.


In [2]:
!git checkout gemini

M	backend/models.py
Already on 'gemini'
Your branch is up to date with 'origin/gemini'.


In [3]:
!git branch -a
!git status

* gemini
  main
  remotes/origin/HEAD -> origin/main
  remotes/origin/backend
  remotes/origin/frontend
  remotes/origin/gemini
  remotes/origin/main
  remotes/origin/uifrontenddesign
On branch gemini
Your branch is up to date with 'origin/gemini'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   backend/models.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	backend/HTF Gemini BackEnd.ipynb
	backend/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [10]:
!cp /content/drive/MyDrive/Colab_Notebooks/HTF_Project/hackthefuture/backend/models.py /content/models.py
!cp /content/drive/MyDrive/Colab_Notebooks/HTF_Project/hackthefuture/backend/config.py /content/config.py

In [8]:
!pip install flask_sqlalchemy
!pip install flask_cors
!pip install flask_jwt_extended
!pip install flask_login

In [12]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GeminiAPI')
client = genai.Client(api_key=API_KEY)

In [18]:
import time
from flask import jsonify
from google.genai import types

def _rank_condition_vid(file_path1: str, file_path2:str=None, file_path3:str=None, file_path4:str=None):
  file = client.files.upload(file=file_path1)
  print("Please wait as we process your video")
  while file.state.name == "PROCESSING":
    print(".", end="")
    time.sleep(5)
    file = client.files.get(name=file.name)

  if file.state.name == "ERROR" or file.state.name == "FAILED":
    print("Error processing Video")

  system_prompt="""
  You are an expert product condition evaluator whose job is to evaluate a
  returned item's condition and categorize it into one of the following: "Like
  New", "Slightly Used", "Used", "Damaged", "Severely Damaged". Respond with the
  category only and nothing else.
  """

  MODEL_ID = "gemini-2.0-flash"
  response = client.models.generate_content(
      model=f"models/{MODEL_ID}",
      contents=["Please evaluate the condition of this returned product and" +
      "categorize it into one of the 5 specified categories.", file],
      config=types.GenerateContentConfig(system_instruction=system_prompt,),
  )

  condition = response.text

  return {"condition": condition}

In [28]:
rank_condition_vid("/content/HTF_gemini_test_image.jpg")

Please wait as we process your video


{'condition': 'Severely Damaged'}

In [19]:
from flask import jsonify
from models import Client, Order
def _process_return_helper(order_id, client_id, file_path1: str, file_path2:str=None,
                   file_path3:str=None, file_path4:str=None):
  condition = rank_condition_vid(file_path1, file_path2, file_path3, file_path4)
  reason = Order.query.filter_by(id=order_id).first().reason

  all_orders = Order.query.filter_by(client_id=client_id).all().count()
  num_returns = Order.query.filter_by(client_id=client_id, returned="Yes").all().count()
  return_rate = num_returns/all_orders

  s = ""

  if return_rate > 0.7:
    s="""Return Denied: You have returned too many items recently. To prevent
    excessive returns, we will temporarily deny return requests.
    """
    return s
  elif return_rate > 0.5:
    s="""Return Denied: You have returned a lot of items recently. If too many
    items from your orders are being returned, we may deny any further returns.
    """
    return s

  # need to check if return is within return period outlined by client policy

  if condition == "Like New" or condition == "Slightly Used":
    if reason == "Wrong Size":
      return "Exchange"
    elif reason in ["Wrong Item", "Item not as described", "Wrong Color",
                    "Not Satisfied", "Dont Want Item Anymore"]:
      return "Restock"
    # need to calcualte
    elif reason in ["Defective or Damaged", "Missing Parts or Accessories"]:
      return "Refurbish"
    else:
      return "Refurbish"
  elif condition == "Used":
    if reason == "Wrong Size":
      return "Exchange"
    elif reason in ["Wrong Item", "Item not as described", "Wrong Color",
                    "Not Satisfied", "Dont Want Item Anymore"]:
      return "Refurbish"
    else:
      return "Refurbish"
  elif condition == "Damaged" or condition == "Severely Damaged":
    # need to check if product was not damaged by user
    if reason == "Defective or Damaged":
      return "Recycle"
    else:
      return "Denied"



In [20]:
from datetime import datetime
def process_return(order_id, customer_id, file_path1: str, file_path2:str=None,
                   file_path3:str=None, file_path4:str=None):
  decision = _process_return_helper(order_id, customer_id, file_path1, file_path2, file_path3, file_path4)
  if decision == "Denied":
    return jsonify({"decision": "Denied"}), 404
  else:
    return jsonify({"decision": "Approved", "action": decision, "order_id": order_id, "return_date": datetime.today()}), 200


In [21]:
!git status

On branch gemini
Your branch is up to date with 'origin/gemini'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   backend/models.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	backend/HTF_Gemini_BackEnd.ipynb
	backend/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [23]:
!git config user.email "tejovardhan.raju@gmail.com"
!git config user.name "Tej79"

In [24]:
!git add backend/HTF_Gemini_BackEnd.ipynb
!git commit -m "Finished Gemini logic"

[gemini 9b6f078] Finished Gemini logic
 1 file changed, 1 insertion(+)
 create mode 100644 backend/HTF_Gemini_BackEnd.ipynb


In [25]:
!git pull

remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 23 (delta 12), reused 23 (delta 12), pack-reused 0 (from 0)
Unpacking objects: 100% (23/23), 6.71 KiB | 22.00 KiB/s, done.
From https://github.com/nicolasiovu/hackthefuture
   38fcae0..a8c70d5  uifrontenddesign -> origin/uifrontenddesign
Already up to date.


In [26]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address
